### Read config file

In [ ]:
import numpy as np
import obspy
from obspy import UTCDateTime
from concurrent.futures import ThreadPoolExecutor
import os
import time
from geopy.distance import great_circle
import folium
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl import load_workbook
import yaml
import shutil

In [ ]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib

In [ ]:
with open('a-project.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project/output_FJSJ_16-01/'               # Harmon server
name_project

In [ ]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = dir_project_workspace + name_project
print('dir_project: ', dir_project)

In [ ]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

### parameter settings

In [ ]:
tag = info_basic['tag']
stalistname_all = info_basic['stalistname_all']

### Generat sta_info

In [ ]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = mathlib.GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist() 
lat_stations_all =  stainfo['latitude'].tolist() 
lon_stations_all =  stainfo['longitude'].tolist() 
elevation_stations_all = stainfo['elevation'].tolist()

In [ ]:
sta_info = {}
years = {2014}
for i in range(nsta_all):
    sta_info[stalist_all[i]] = {}
    sta_info[stalist_all[i]]['year'] = 2014
    sta_info[stalist_all[i]]['lat'] =float(lat_stations_all[i])
    sta_info[stalist_all[i]]['lon'] =float(lon_stations_all[i])

### Sort by different years

In [ ]:
cs = maplib.cs
crs = maplib.crs
zoom_rate = 16

In [ ]:
key_loc = 12
this_map = maplib.plot_single_area(sta_info,key_loc,cs,crs,zoom_rate)
this_map

### Sort area automatically with row and column indexes

In [ ]:
def sort_point_2d(sta_info,stas,step_col,step_row,size_col,size_row,tag_col,tag_row):
    rows = np.arange(int(tag_row[0]),int(tag_row[1])+1)
    cols = np.arange(int(tag_col[0]),int(tag_col[1])+1)

    len_row = int((size_row-1)/2)
    len_col = int((size_col-1)/2)

    sta_area = {}
    flag_row = tag_row[0]

    while flag_row + len_row <= tag_row[1]:
        while flag_row - len_row <=  tag_row[0]-1:
            flag_row += 1
        flag_col = tag_col[0]
        while flag_col + len_col <= tag_col[1]:
            while flag_col - len_col <= tag_col[0]-1:
                flag_col += 1
            
            key = str(flag_col).zfill(2)+'-'+str(flag_row).zfill(2)
            
            sta_area[key] = {}
            for i in range(flag_row-len_row-1,flag_row+len_row):
                for j in range(flag_col-len_col-1,flag_col+len_col):
                    name = 'R'+str(cols[j]).zfill(2)+str(rows[i]).zfill(2)
                    if name in stas:
                        sta_area[key][name] ={}
                        sta_area[key][name]['lat'] = sta_info[name]['lat']
                        sta_area[key][name]['lon'] = sta_info[name]['lon']
            flag_col += step_col
        flag_row += step_row
    return sta_area

In [ ]:
step_row = 1
step_col = 1

size_row = 7
size_col = 19

tag_row = [1,20]
tag_col = [1,60]

stas = list(sta_info.keys())

sta_area = sort_point_2d(sta_info,stas,step_col,step_row,size_col,size_row,tag_col,tag_row)

In [ ]:
this_map = maplib.plot_single_area(sta_info,0,cs,crs,zoom_rate)
i = 2
key = list(sta_area.keys())[i]
#key = '13-04'
this_map = maplib.lat_lon(sta_area[key],cs[np.mod(i+1,2)],crs[np.mod(i,2)],this_map)
this_map

In [ ]:
this_map = maplib.plot_single_area(sta_info,10,cs,crs,zoom_rate)
for i in range(len(sta_area.keys())):
    key = list(sta_area.keys())[i]
    this_map = maplib.lat_lon(sta_area[key],cs[np.mod(i,4)],crs[np.mod(i+1,3)],this_map)
this_map

### Generate subworks

In [ ]:
key_subworks = list(sta_area.keys())
info_basic['key_subworks'] = list(sta_area.keys())
len(key_subworks)

In [ ]:
dir_partition = dir_project + 'partition/'
info_basic['rdir_partition'] = 'partition/'
if os.path.exists(dir_partition):
    shutil.rmtree(dir_partition)
os.makedirs(dir_partition)

In [ ]:
for key_subwork in key_subworks:
    Station = list(sta_area[key_subwork].keys())
    lat = [sta_area[key_subwork][key]['lat'] for key in list(sta_area[key_subwork].keys())]
    lon = [sta_area[key_subwork][key]['lon'] for key in list(sta_area[key_subwork].keys())]
    filename = dir_partition + key_subwork + '.txt'
    np.savetxt(filename, np.array([Station, lat, lon]).T, fmt='%s', delimiter=' ', header='Station lat lon')

In [ ]:
#np.save(info_basic['dir_config']+'/Basic_info.npy', info_basic)
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)